In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

class SimpleRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        return self.fc(out)

# Prepare dataset for auto-regression (predict next value)
def prepare_autoregression_data(seq_len=50, pred_len=1):
    x = np.linspace(0, 4 * np.pi, seq_len)
    y = np.sin(x)
    data = torch.tensor(y, dtype=torch.float32).view(-1, 1)
    
    X, Y = [], []
    for i in range(len(data) - pred_len):
        X.append(data[i:i+pred_len])
        Y.append(data[i+pred_len])
    
    return torch.stack(X), torch.stack(Y)

X_train, Y_train = prepare_autoregression_data()

# Train auto-regression model
def train_autoregressor(X, Y, epochs=200, lr=0.005):
    model = SimpleRNN(1, 16)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(X.unsqueeze(-1))[:, -1]  # Predict last step
        loss = loss_fn(output, Y)
        loss.backward()
        optimizer.step()
        
        if epoch % 50 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
    
    return model

# Train and visualize results
auto_model = train_autoregressor(X_train, Y_train)

# Predict next values
preds = auto_model(X_train.unsqueeze(-1)).detach().numpy()
plt.plot(Y_train.numpy(), label="True")
plt.plot(preds[:, -1], label="Predicted")
plt.legend()
plt.show()